# Standard packages

In [ ]:
import os
import pandas as pd
import time

# Specific installs

RNAShapes is available as an Anaconda package.

**Note:** if the colab session fails (with unkown reason), run all the cells again. It should work ok the second time. 

In [ ]:
method_name = "RNAshapes"

In [ ]:
# install Anaconda
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


In [ ]:
# install package
!conda install -c bioconda rnashapes --yes > /dev/null
!RNAshapes --help | grep "^#" | grep "version"

#            version 3.3.2 (21.04.2021)


# S. cerevisiae (sce) 18 long non-coding RNA dataset
Data source: https://genie.weizmann.ac.il/pubs/PARS10/data/sce_genes_folded.tab.gz

In [ ]:
gh_path = "https://raw.githubusercontent.com/sinc-lab/lncRNA-folding/master/data/"
sce = pd.read_csv(gh_path + "sce_genes_folded.tab", delimiter='\t', 
                  header=None, index_col=0, 
                  names=("Gene ID", "sequence", "PARS-assisted folding"))

In [ ]:
# Sequences to process
yeast18lnc = ["snR81", "snR34", "snR43", "snR44",  "snR31",  "snR10",
              "snR63", "snR11", "snR82", "snR17b", "snR17a", "snR37",
              "SCR1",  "SRG1",  "snR19", "snR30",  "LSR1",   "TLC1"]

# Compute structures

In [9]:
def run_folding(fasta_name, temp=37):
  out_file_name = fasta_name + '.out'

  # make prediction and clean output file
  os.system(f"RNAshapes -mode mfe --temperature {temp} {fasta_name} 2>/dev/null | \
   sed 's/^ \+//g' | \
   sed 's/ \+/\t/g' | \
   cut -f2 > {out_file_name}")

  return out_file_name

In [10]:
out_fasta_name = method_name + "_yeast18"
if os.path.exists(out_fasta_name + ".fasta"): os.remove(out_fasta_name + ".fasta")

lnc_ids = yeast18lnc
print("   \t lnc \t len \t time")
for i, lnc in enumerate(lnc_ids): 

  start_time = time.time()
  seq = sce.loc[lnc]["sequence"]
  print(f"{i+1}/{len(lnc_ids)}\t{lnc} \t {len(seq)}", end='\t')

  # Write a one-sequence fasta
  with open("tmp.fasta", "w") as ofile: 
    ofile.write(f">{lnc}\n{seq}\n")
  
  dot_file_name = run_folding("tmp.fasta")

  # Concatenate outputs
  os.system("cat " + dot_file_name + " >> " + out_fasta_name + ".fasta") 

  print(f"{time.time() - start_time: .1f} s")

   	 lnc 	 len 	 time
1/18	snR81 	 201	 0.8 s
2/18	snR34 	 203	 0.8 s
3/18	snR43 	 209	 0.8 s
4/18	snR44 	 211	 0.8 s
5/18	snR31 	 225	 1.0 s
6/18	snR10 	 245	 1.2 s
7/18	snR63 	 255	 1.4 s
8/18	snR11 	 258	 1.5 s
9/18	snR82 	 268	 1.6 s
10/18	snR17b 	 332	 3.0 s
11/18	snR17a 	 333	 3.0 s
12/18	snR37 	 386	 4.8 s
13/18	SCR1 	 522	 12.7 s
14/18	SRG1 	 551	 14.3 s
15/18	snR19 	 568	 16.0 s
16/18	snR30 	 606	 19.8 s
17/18	LSR1 	 1175	 174.5 s
18/18	TLC1 	 1301	 233.5 s
